In [14]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import numpy as np
import os
import cv2
import random
from PIL import Image

# Definición de variables y lectura de datos
path_test = "/content/drive/MyDrive/DatasetClasificador"
CATEGORIES = ["Regla", "Sin_Regla"]  # Modificado para la tarea binaria

IMG_SIZE = 200
training = []

def createTrainingData():
    for category in CATEGORIES:
        path = os.path.join(path_test, category)
        class_num = CATEGORIES.index(category)
        for img_name in os.listdir(path):
            img_path = os.path.join(path, img_name)
            img_array = cv2.imread(img_path)
            new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
            training.append([new_array, class_num])

createTrainingData()
random.shuffle(training)
features, labels = zip(*training)
X = []
y = []

for feature, label in zip(features, labels):
    X.append(feature)
    y.append(label)

X = np.array(X).reshape(-1, IMG_SIZE, IMG_SIZE, 3)
X = X.astype('float32')
X /= 255

Y = to_categorical(y, num_classes=2)  # Modificado para la tarea binaria

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=4)

batch_size = 16
nb_epochs = 25
img_rows, img_columns = 200, 200
img_channels = 3
nb_filters = 32
nb_pool = 2
nb_conv = 3

# Modelo
model = Sequential([
    Conv2D(32, (3,3), padding='same', activation='relu',
           input_shape=(200, 200, 3)),
    MaxPooling2D((2, 2), strides=2),
    Conv2D(32, (3,3), padding='same', activation='relu'),
    MaxPooling2D((2, 2), strides=2),
    Dropout(0.5),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(2, activation='softmax')  # Modificado para la tarea binaria
])

model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

# Entrenamiento
model.fit(X_train, y_train, batch_size=batch_size, epochs=nb_epochs, verbose=1, validation_data=(X_test, y_test))

# Evaluación
score = model.evaluate(X_test, y_test, verbose=0)

print("Test accuracy: ", score[1])


Epoch 1/25
28/28 [==============================] - 31s 1s/step - loss: 1.3466 - accuracy: 0.5045 - val_loss: 0.6935 - val_accuracy: 0.5179
Epoch 2/25
28/28 [==============================] - 29s 1s/step - loss: 0.6938 - accuracy: 0.5045 - val_loss: 0.6938 - val_accuracy: 0.4375
Epoch 3/25
28/28 [==============================] - 28s 1s/step - loss: 0.6929 - accuracy: 0.5045 - val_loss: 0.6937 - val_accuracy: 0.4554
Epoch 4/25
28/28 [==============================] - 28s 988ms/step - loss: 0.6930 - accuracy: 0.5067 - val_loss: 0.6946 - val_accuracy: 0.4554
Epoch 5/25
28/28 [==============================] - 28s 988ms/step - loss: 0.6941 - accuracy: 0.5112 - val_loss: 0.6946 - val_accuracy: 0.4643
Epoch 6/25
28/28 [==============================] - 30s 1s/step - loss: 0.6926 - accuracy: 0.5223 - val_loss: 0.6931 - val_accuracy: 0.4732
Epoch 7/25
28/28 [==============================] - 29s 1s/step - loss: 0.6855 - accuracy: 0.5424 - val_loss: 0.6793 - val_accuracy: 0.4464
Epoch 8/25
28/

In [15]:
from tensorflow.keras.models import load_model

# Ruta al archivo del modelo guardado
modelo_archivo = '/content/drive/MyDrive/classifier.keras'

# Carga el modelo
modelo_cargado = load_model(modelo_archivo)


In [25]:
import cv2

# Ruta a la imagen
imagen_ruta = '/content/drive/MyDrive/DatasetClasificador/Regla/input_103.jpg'

# Leer la imagen
imagen = cv2.imread(imagen_ruta)

# Redimensionar la imagen a la dimensión esperada por el modelo
# Asume que tu modelo espera imágenes de tamaño (200, 200, 3)
imagen_redimensionada = cv2.resize(imagen, (200, 200))

# Normalizar la imagen (si tu modelo requiere que los datos estén normalizados)
imagen_normalizada = imagen_redimensionada / 255.0


In [26]:
# Expandir la dimensión de la imagen
imagen_expandida = np.expand_dims(imagen_normalizada, axis=0)


In [27]:
# Obtener la predicción
prediccion = modelo_cargado.predict(imagen_expandida)

# La predicción será una matriz de probabilidades para cada clase
# Puedes obtener la clase predicha con el índice de la máxima probabilidad
clase_predicha = np.argmax(prediccion)

print(f"Clase predicha: {clase_predicha}")


1/1 [==============================] - 0s 36ms/step
Clase predicha: 0
